In [518]:
# I will use this cell to import every library that i need.
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns 
import re

In [519]:
# in order to import the data, I need to know what kind of encoding my file has.

with open('data/attacks.csv') as attackscsv:
    print(attackscsv)

<_io.TextIOWrapper name='data/attacks.csv' mode='r' encoding='cp1252'>


In [520]:
# I open the file and check the headings.

attacks = pd.read_csv('data/attacks.csv', encoding='cp1252')
attacks.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,...,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,...,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,...,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,...,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


In [521]:
# I will check basic info and then  drop all the rows or columns that have no values or only 1.
def BasicCleanig (dataframe,NumberOfValuesInRow,NumberOfValuesInColum):
    '''This function recevies a DataFrame, the minimum number of values that I want in my rows and columns.
        It deletes the empty rows, the empty columns, and the duplicates.
        Returns the DataFrame with the changes aplied.'''
    dataframe.dropna(axis=1, thresh= NumberOfValuesInColum, inplace=True)
    dataframe.dropna(axis=0, thresh=NumberOfValuesInRow, inplace=True)
    dataframe.drop_duplicates(inplace=True)
    return dataframe

attacks = BasicCleanig(attacks,5,5)


In [522]:
# The columns Date and Year contain data that is not constistent. The case number has the correct date, but in a weird format. I will get it with regex. Using (\d{4})\.(\d{2})\.(\d{2})

attacks.rename(columns={'Year':'OldYear'},inplace=True)
attacks[['Year','Month','Day']]= attacks['Case Number'].str.extract(r'(\d{4})\.(\d{2})\.(\d{2})')
attacks.drop(columns=['Case Number','Date','OldYear','Case Number.1','Case Number.2','original order', 'href'], inplace=True)



In [529]:
# I defined a functions to be able to group related values
def activities (x):
    ''' This function receives a field, it searchs for different key words and returns the group it belongs'''
    string = str(x).lower()
    if 'swim' in string or 'bath' in string or 'div' in string or 'snork' in string or 'feed' in string or 'wading' in string or 'shark' in string or 'air' in string or 'wash' in string or 'walk' in string or 'floa' in string or 'tread' in string or 'water' in string or 'play' in string:
        return 'swiming'
    elif 'surf'  in string or 'board' in string:
        return 'surfing'
    elif 'fish' in string or 'stan' in string:
        return 'fishing'
    elif 'ship' in string or 'boat' in string or 'sail' in string or 'kaya' in string or 'cano' in string or 'adrift' in string or 'pad' in string or 'sunk' in string or 'yatch' in string or 'yacht' in string or 'row'in string or 'sink' in string or 'sea' in string or 'fell' in string or 'fall' in string:
        return 'sailing'
    else:
        return 'others'
def daytime (x):
    ''' this function receives a string and returns the moment of the day'''
    x=str(x)
    match= re.match(r'(\d{2})h\d{2}?', x)
    if match:
        hourint = int(match.group(1))
        if hourint > 6 and hourint < 12:
            return 'morning'
        elif hourint >= 12 and hourint < 21:
            return 'afternoon'
        elif hourint <= 6 or hourint >= 21:
            return 'night'
    else:
        if 'noon' in x or 'sunset' in x:
            return 'afternoon'
        elif 'night' in x:
            return 'night'
        elif 'morning' in x or 'day' in x or 'sunrise' in x:
            return 'morning'
        else:
            return'unknown'
def agecorrect (x):
    '''This function looks for the correct format of an age, otherwise returns unknonw'''
    x=str(x)
    match= re.match(r'(\d{1,2})', x)
    if match:
        ageint = int(match.group(1))
        return ageint
    else:
        return ''
def fatalattack (x):
    ''' This function looks for the exact match of the letters Y and N'''
    x=str(x).upper()
    match = re.match(r"(^[N,Y]$)",x)
    if match:
        fatalstr = (match.group(1))
        return fatalstr
    else:
        return 'unknown'
        
def speciesshark (x):
    ''' This function searches for keywords and return the species'''
    x=str(x).lower()
    if 'white' in x:
        return 'white shark'
    elif 'tiger' in x:
        return 'tiger shark'
    elif 'bull' in x:
        return 'bull shark'
    elif 'wobbegong'in x:
        return 'wobbegong shark'
    elif 'ham' in x:
        return 'hammerhead shark'
    elif 'confirma' in x or 'invalid' in x or 'quest' in x:
        return'no shark involved'
    
    else:
        return 'species not confirmed'
    

  
attacks.columns=attacks.columns.str.lstrip()
attacks.columns=attacks.columns.str.rstrip()
attacks.head()
attacks['Sex']= attacks['Sex'].apply(lambda x : 'F' if x == 'F' else 'M')
attacks['Type']= attacks['Type'].apply(lambda x : 'Boat' if x=='Boating' or x=='Boatomg' or x == 'Questionable' else x)
attacks['Activity']= attacks['Activity'].apply(activities)
attacks['Time']= attacks['Time'].apply(daytime)
attacks['Age']=attacks['Age'].apply(agecorrect)
attacks['FataFatal (Y/N)'] = attacks['Fatal (Y/N)'].apply(fatalattack)
attacks['Species']=attacks['Species'].apply(speciesshark)
attacks.to_csv('data/attacks_clean.csv')

In [524]:
for i in attacks.columns:
    print(f'In the column {i} \n',attacks[i].value_counts(),'\n')

In the column Type 
 Unprovoked      4595
Provoked         574
Invalid          547
Boat             343
Sea Disaster     239
Name: Type, dtype: int64 

In the column Country 
 USA                       2229
AUSTRALIA                 1338
SOUTH AFRICA               579
PAPUA NEW GUINEA           134
NEW ZEALAND                128
                          ... 
MALDIVE ISLANDS              1
NICARAGUA                    1
NORTH SEA                    1
RED SEA / INDIAN OCEAN       1
CEYLON (SRI LANKA)           1
Name: Country, Length: 212, dtype: int64 

In the column Area 
 Florida                                 1037
New South Wales                          486
Queensland                               311
Hawaii                                   298
California                               290
                                        ... 
Ysabel Island                              1
 Lau Province                              1
South Coast, East New Britain              1
Between South